In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
from torch.utils.data import Dataset, DataLoader

In [2]:
from os import listdir
from os.path import join
import random

from PIL import Image
import torch
import torch.utils.data as data
import torchvision.transforms as transforms

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])


def load_img(filepath):
    img = Image.open(filepath).convert('RGB')
    img = img.resize((256, 256), Image.BICUBIC)
    return img


def save_img(image_tensor, filename):
    image_numpy = image_tensor.float().numpy()
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
    image_numpy = image_numpy.clip(0, 255)
    image_numpy = image_numpy.astype(np.uint8)
    image_pil = Image.fromarray(image_numpy)
    image_pil.save(filename)
    print("Image saved as {}".format(filename))


class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, direction):
        super(DatasetFromFolder, self).__init__()
        self.direction = direction
        self.a_path = join(image_dir, "a")
        self.b_path = join(image_dir, "b")
        self.image_filenames = [x for x in listdir(self.a_path) if is_image_file(x)]

        transform_list = [transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]

        self.transform = transforms.Compose(transform_list)

    def __getitem__(self, index):
        a = Image.open(join(self.a_path, self.image_filenames[index])).convert('RGB')
        b = Image.open(join(self.b_path, self.image_filenames[index])).convert('RGB')
        a = a.resize((286, 286), Image.BICUBIC)
        b = b.resize((286, 286), Image.BICUBIC)
        a = transforms.ToTensor()(a)
        b = transforms.ToTensor()(b)
        w_offset = random.randint(0, max(0, 286 - 256 - 1))
        h_offset = random.randint(0, max(0, 286 - 256 - 1))
    
        a = a[:, h_offset:h_offset + 256, w_offset:w_offset + 256]
        b = b[:, h_offset:h_offset + 256, w_offset:w_offset + 256]
    
        a = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(a)
        b = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(b)

        if random.random() < 0.5:
            idx = [i for i in range(a.size(2) - 1, -1, -1)]
            idx = torch.LongTensor(idx)
            a = a.index_select(2, idx)
            b = b.index_select(2, idx)

        if self.direction == "a2b":
            return a, b
        else:
            return b, a

    def __len__(self):
        return len(self.image_filenames)
    

def get_training_set(root_dir, direction):
    train_dir = join(root_dir, "train")
    return DatasetFromFolder(train_dir, direction)

def get_validation_set(root_dir, direction):
    val_dir = join(root_dir, "val")
    return DatasetFromFolder(val_dir, direction)

def get_testing_set(root_dir, direction):
    test_dir = join(root_dir, "test")
    return DatasetFromFolder(test_dir, direction)

In [3]:
# Test data
root_path='/scratch/datasets/pose2video/s11/'
test_set = get_testing_set(root_path, 'a2b')
test_data_loader = DataLoader(dataset=test_set, num_workers=4, batch_size=8, shuffle=True)

In [4]:
def plot_test_result(input, target, gen_image, epoch, training=True, save=False, save_dir='results/', show=False, fig_size=(5, 5)):
    if not training:
        fig_size = (input.size(2) * 3 / 100, input.size(3)/100)

    fig, axes = plt.subplots(1, 3, figsize=fig_size)
    imgs = [input, gen_image, target]
    for ax, img in zip(axes.flatten(), imgs):
        ax.axis('off')
        ax.set_adjustable('box')
        # Scale to 0-255
        img = (((img[0] - img[0].min()) * 255) / (img[0].max() - img[0].min())).numpy().transpose(1, 2, 0).astype(np.uint8)
        ax.imshow(img, cmap=None, aspect='equal')
    plt.subplots_adjust(wspace=0, hspace=0)

    if training:
        title = 'Epoch {0}'.format(epoch + 1)
        fig.text(0.5, 0.04, title, ha='center')

    # save figure
    if save:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        if training:
            save_fn = save_dir + 'Result_epoch_{:d}'.format(epoch+1) + '.png'
        else:
            save_fn = save_dir + 'Test_result_{:d}'.format(epoch+1) + '.png'
            fig.subplots_adjust(bottom=0)
            fig.subplots_adjust(top=1)
            fig.subplots_adjust(right=1)
            fig.subplots_adjust(left=0)
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()

In [5]:
class ConvBlock(torch.nn.Module):
    def __init__(self, input_size, output_size, kernel_size=4, stride=2, padding=1, activation=True, batch_norm=True):
        super(ConvBlock, self).__init__()
        self.conv = torch.nn.Conv2d(input_size, output_size, kernel_size, stride, padding)
        self.activation = activation
        self.lrelu = torch.nn.LeakyReLU(0.2, True)
        self.batch_norm = batch_norm
        self.bn = torch.nn.BatchNorm2d(output_size)

    def forward(self, x):
        if self.activation:
            out = self.conv(self.lrelu(x))
        else:
            out = self.conv(x)

        if self.batch_norm:
            return self.bn(out)
        else:
            return out


class DeconvBlock(torch.nn.Module):
    def __init__(self, input_size, output_size, kernel_size=4, stride=2, padding=1, batch_norm=True, dropout=False):
        super(DeconvBlock, self).__init__()
        self.deconv = torch.nn.ConvTranspose2d(input_size, output_size, kernel_size, stride, padding)
        self.bn = torch.nn.BatchNorm2d(output_size)
        self.drop = torch.nn.Dropout(0.5)
        self.relu = torch.nn.ReLU(True)
        self.batch_norm = batch_norm
        self.dropout = dropout

    def forward(self, x):
        if self.batch_norm:
            out = self.bn(self.deconv(self.relu(x)))
        else:
            out = self.deconv(self.relu(x))

        if self.dropout:
            return self.drop(out)
        else:
            return out
        
class Generator(torch.nn.Module):
    def __init__(self, input_dim, num_filter, output_dim):
        super(Generator, self).__init__()

        # Encoder
        self.conv1 = ConvBlock(input_dim, num_filter, activation=False, batch_norm=False)
        self.conv2 = ConvBlock(num_filter, num_filter * 2)
        self.conv3 = ConvBlock(num_filter * 2, num_filter * 4)
        self.conv4 = ConvBlock(num_filter * 4, num_filter * 8)
        self.conv5 = ConvBlock(num_filter * 8, num_filter * 8)
        self.conv6 = ConvBlock(num_filter * 8, num_filter * 8)
        self.conv7 = ConvBlock(num_filter * 8, num_filter * 8)
        self.conv8 = ConvBlock(num_filter * 8, num_filter * 8, batch_norm=False)
        # Decoder
        self.deconv1 = DeconvBlock(num_filter * 8, num_filter * 8, dropout=True)
        self.deconv2 = DeconvBlock(num_filter * 8 * 2, num_filter * 8, dropout=True)
        self.deconv3 = DeconvBlock(num_filter * 8 * 2, num_filter * 8, dropout=True)
        self.deconv4 = DeconvBlock(num_filter * 8 * 2, num_filter * 8)
        self.deconv5 = DeconvBlock(num_filter * 8 * 2, num_filter * 4)
        self.deconv6 = DeconvBlock(num_filter * 4 * 2, num_filter * 2)
        self.deconv7 = DeconvBlock(num_filter * 2 * 2, num_filter)
        self.deconv8 = DeconvBlock(num_filter * 2, output_dim, batch_norm=False)

    def forward(self, x):
        # Encoder
        enc1 = self.conv1(x)
        enc2 = self.conv2(enc1)
        enc3 = self.conv3(enc2)
        enc4 = self.conv4(enc3)
        enc5 = self.conv5(enc4)
        enc6 = self.conv6(enc5)
        enc7 = self.conv7(enc6)
        enc8 = self.conv8(enc7)
        # Decoder with skip-connections
        dec1 = self.deconv1(enc8)
        dec1 = torch.cat([dec1, enc7], 1)
        dec2 = self.deconv2(dec1)
        dec2 = torch.cat([dec2, enc6], 1)
        dec3 = self.deconv3(dec2)
        dec3 = torch.cat([dec3, enc5], 1)
        dec4 = self.deconv4(dec3)
        dec4 = torch.cat([dec4, enc4], 1)
        dec5 = self.deconv5(dec4)
        dec5 = torch.cat([dec5, enc3], 1)
        dec6 = self.deconv6(dec5)
        dec6 = torch.cat([dec6, enc2], 1)
        dec7 = self.deconv7(dec6)
        dec7 = torch.cat([dec7, enc1], 1)
        dec8 = self.deconv8(dec7)
        out = torch.nn.Tanh()(dec8)
        return out

    def normal_weight_init(self, mean=0.0, std=0.02):
        for m in self.children():
            if isinstance(m, ConvBlock):
                torch.nn.init.normal(m.conv.weight, mean, std)
            if isinstance(m, DeconvBlock):
                torch.nn.init.normal(m.deconv.weight, mean, std)
                
class Discriminator(torch.nn.Module):
    def __init__(self, input_dim, num_filter, output_dim):
        super(Discriminator, self).__init__()

        self.conv1 = ConvBlock(input_dim, num_filter, activation=False, batch_norm=False)
        self.conv2 = ConvBlock(num_filter, num_filter * 2)
        self.conv3 = ConvBlock(num_filter * 2, num_filter * 4)
        self.conv4 = ConvBlock(num_filter * 4, num_filter * 8, stride=1)
        self.conv5 = ConvBlock(num_filter * 8, output_dim, stride=1, batch_norm=False)

    def forward(self, x, label):
        x = torch.cat([x, label], 1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        out = torch.nn.Sigmoid()(x)
        return out

    def normal_weight_init(self, mean=0.0, std=0.02):
        for m in self.children():
            if isinstance(m, ConvBlock):
                torch.nn.init.normal(m.conv.weight, mean, std)

In [6]:
G = torch.load('/scratch/abi/MultiSign/ckpt/G-4-7.232555389404297')
D = torch.load('/scratch/abi/MultiSign/ckpt/D-4-0.1802155077457428')
G.cuda()
D.cuda()

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/abi/MultiSign/ckpt/G-4-7.232555389404297'

In [8]:
G.eval()

Generator(
  (conv1): ConvBlock(
    (conv): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): ConvBlock(
    (conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): ConvBlock(
    (conv): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_ru

In [9]:
MSE = torch.nn.MSELoss()
L1 = torch.nn.L1Loss()

In [10]:
MSE = torch.nn.MSELoss()
L1 = torch.nn.L1Loss()
l1 = 0
psnr = 0
for i, (input, target) in enumerate(test_data_loader):
    # input & target image data
    x_ = Variable(input.cuda())
    y_ = Variable(target.cuda())

    gen_image = G(x_)
    gen_image = gen_image.cpu().data
    
    l1 = l1 + L1(target, gen_image)
    psnr = psnr + 20 * torch.log10(255.0 / torch.sqrt(MSE(target, gen_image)))

    if i < 20:
        # Show result for test data
        plot_test_result(input, target, gen_image, i, training=False, save=True, save_dir='/scratch/abi/MultiSign/fun/')
        print('%d images are generated.' % (i + 1))
    
l1_loss = l1 / len(test_data_loader)
psnr_loss = psnr / len(test_data_loader)
print("L1: ", l1_loss)
print("PSNR: ", psnr_loss)

torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
1 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
2 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
3 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
4 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
5 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
6 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
7 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
8 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
9 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
10 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
11 images are generated.
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
12 images are generated.
torch.Size([8

torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8, 3, 256, 256]) torch.Size([8, 3, 256, 256])
torch.Size([8,